In [1]:
from kafka_cffi import Producer as cffi_Producer
from confluent_kafka import Producer as confluent_Producer
import time
import platform

conf = {
    "bootstrap.servers": "n1.kafka10.service.consul:9092",
    "queue.buffering.max.messages": "1000000",
    "queue.buffering.max.kbytes": "2097151",
    "acks": "1",
#     "compression.codec": "snappy",
}

cffi_prod = cffi_Producer(conf)
confluent_prod = confluent_Producer(conf)

def callback(err, msg):
    pass

def benchmark(prod, samples, per_batch, msg_length=100, on_delivery=None):
    print(type(prod))
    timings = []
    for s in range(samples):
        t = time.time()
        for i in range(per_batch):
            prod.produce("moo", "0" * msg_length, on_delivery=on_delivery)
        prod.flush(-1)
        timings.append(time.time() - t)
    
    print("%d Samples, %d msgs per batch, %d bytes/msg" % (samples, per_batch, msg_length))
    print("Mean (per batch): %0.3f msgs/sec" % (sum(per_batch / x for x in timings) / samples))
    print("Mean (overall): %0.3f msgs/sec" % (samples * per_batch / sum(timings)))
    print("Fastest batch: %0.3f msgs/sec" % (per_batch / min(timings)))
    print("Slowest batch: %0.3f msgs/sec" % (per_batch / max(timings)))
    
print(platform.python_implementation(), platform.python_version())

('CPython', '2.7.12')


In [2]:
for i in range(1000):
    # warm cold topic up first for both clients to be fair
    cffi_prod.produce("moo", "hello world")
    confluent_prod.produce("moo", "hello world")
    
cffi_prod.flush()
confluent_prod.flush()

0L

In [3]:
benchmark(cffi_prod, samples=30, per_batch=100000, msg_length=100)
benchmark(confluent_prod, samples=30, per_batch=100000, msg_length=100)

<class 'kafka_cffi.producer.Producer'>
30 Samples, 100000 msgs per batch, 100 bytes/msg
Mean (per batch): 298781.690 msgs/sec
Mean (overall): 284922.807 msgs/sec
Fastest batch: 402741.981 msgs/sec
Slowest batch: 178959.719 msgs/sec
<type 'cimpl.Producer'>
30 Samples, 100000 msgs per batch, 100 bytes/msg
Mean (per batch): 383659.838 msgs/sec
Mean (overall): 358810.563 msgs/sec
Fastest batch: 540417.434 msgs/sec
Slowest batch: 211921.836 msgs/sec


In [4]:
benchmark(cffi_prod, samples=60, per_batch=20000, msg_length=450)
benchmark(confluent_prod, samples=60, per_batch=20000, msg_length=450)

<class 'kafka_cffi.producer.Producer'>
60 Samples, 20000 msgs per batch, 450 bytes/msg
Mean (per batch): 165611.219 msgs/sec
Mean (overall): 152869.169 msgs/sec
Fastest batch: 308655.488 msgs/sec
Slowest batch: 72749.764 msgs/sec
<type 'cimpl.Producer'>
60 Samples, 20000 msgs per batch, 450 bytes/msg
Mean (per batch): 228022.150 msgs/sec
Mean (overall): 215962.537 msgs/sec
Fastest batch: 414293.165 msgs/sec
Slowest batch: 122876.520 msgs/sec


In [5]:
# performance with a noop callback
benchmark(cffi_prod, samples=60, per_batch=20000, msg_length=450, on_delivery=callback)
benchmark(confluent_prod, samples=60, per_batch=20000, msg_length=450, on_delivery=callback)

<class 'kafka_cffi.producer.Producer'>
60 Samples, 20000 msgs per batch, 450 bytes/msg
Mean (per batch): 115232.191 msgs/sec
Mean (overall): 109551.825 msgs/sec
Fastest batch: 171825.876 msgs/sec
Slowest batch: 65738.402 msgs/sec
<type 'cimpl.Producer'>
60 Samples, 20000 msgs per batch, 450 bytes/msg
Mean (per batch): 226061.302 msgs/sec
Mean (overall): 216108.982 msgs/sec
Fastest batch: 454329.739 msgs/sec
Slowest batch: 123298.420 msgs/sec
